In [ ]:
!pip install openai
!pip install tiktoken

In [ ]:
# @title Relevant paths for csv-files stored on my google drive
path_reviews = "./drive/MyDrive/datasets/semotion-2017/reviews.csv"
path_reviews_extended = "/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/reviews_extended.csv"
path_appStore = "/content/drive/MyDrive/datasets/AppleStore.csv"
path_selected_reviews="/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/selected_reviews.csv"
path_selected_reviews_35k="/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/selected_reviews_35k.csv"
path_selected_reviews_28k = "/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/selected_reviews_28k.csv"
path_selected_reviews_19k = "/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/selected_reviews_19k.csv"
path_selected_reviews_10k = "/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/selected_reviews_10k.csv"
path_annotated_reviews = "/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/annotated_reviews.csv"
path_distilled_absa="/content/drive/MyDrive/Python/distilling-step-by-step-main/distilling-step-by-step-main/datasets/absa"
path_app_names = "/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/app_names.json"
path_app_categories = "/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/app_categories.json"
path_general_aspects = "/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/general_aspects.json"
path_category_specific_aspects = "/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/category_specfic_aspects.json"
path_generated_dataset_names = "/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/generated_dataset_names.json"
path_allowed_models = "/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/allowed_models.json"
path_filtered_and_shuffled = "/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/filtered_and_shuffled_reviews_35k.csv"
MODEL = 't5-large'
OUT_DIR = '/content/drive/MyDrive/Python/Notebooks/Masters_Thesis/models_t5'
data_types = {'appId' : int,
              'body': str,
              'date': str,
              'edited': str,
              'name': str,
              'rating': float,
              'sentComb': float,
              'sentNeg': float,
              'sentPos': float,
              'title': str,
              'userReviewId': str,
              'viewUsersUserReviewsUrl': str,
              'voteCount': float,
              'voteSum': int,
              'titleBody': str,
              'appName': str,
              'appCategory': str,
              'tokenCount': int,
              'numberOfReviews': int,
              'tiktokenInputTokens': int,
              'T5-tokens' : int,
              'aspectsAndScores': str,
              'priceInfo' : str,
              'ABSALabels' : str,
              'ABSARationales' : str,
              'best_24_05_07_t5' : str,
              'ABSALabels-T5-tokens' : int,
              'ABSARationales-T5-tokens' : int}
images_dir = '/content/drive/MyDrive/sentiments_images'

In [ ]:
# @title The Review class (inherits from pydantic BaseModel and has methods for validation)
Review_meta_data = {path_app_names : [], path_app_categories : [], path_general_aspects : [], path_category_specific_aspects : [], path_allowed_models : [], path_generated_dataset_names : []}
for path in Review_meta_data:
  with open(path, 'r') as _f:
    Review_meta_data[path] = json.load(_f)

class Review(BaseModel):
    user_review_id: str= Field(description="The review Id of the review")
    app_name: str = Field(description="The app name of the app in question")
    title: str = Field(description="The title of the review")
    body: str = Field(description="The body of the review")
    app_category : str = Field(description="The app category of the app in question")
    aspects : List[str] = Field(description="The possible aspects of the review in a list")
    aspects_and_scores: Optional[Dict[str, str]] = Field(description="The result from manual annotation of the review. ", default=None)
    generated_aspects_and_scores: Dict[str, Dict[str, Any]] = Field(description="A dictionary containing the results from openAI API-calls.", default={})
    _valid_sentiments: ClassVar[List[str]]  = ["positive", "negative", "neutral"]
    _allowed_models: ClassVar[List[str]]  = Review_meta_data[path_allowed_models]
    _generated_dataset_names: ClassVar[List[str]] = Review_meta_data[path_generated_dataset_names]
    _unique_app_names: ClassVar[List[str]] = Review_meta_data[path_app_names]
    _unique_app_categories: ClassVar[List[str]] = Review_meta_data[path_app_categories]
    _general_aspects : ClassVar[List[str]] = Review_meta_data[path_general_aspects]
    _category_specific_aspects : ClassVar[Dict[str, List[str]]] = Review_meta_data[path_category_specific_aspects]

    @classmethod
    def from_dataframe_row(cls, row):
        # Assuming the row is a pandas Series
        generated_aspects_and_scores = {column : json.loads(row[column]) if (column not in row.index or pd.isna(row[column]) or row[column]=='')  is False else {} for column in cls._generated_dataset_names}
        return cls(
            user_review_id=row['userReviewId'],
            app_name=row['appName'],
            title=row['title'],
            body=row['body'],
            app_category=row['appCategory'],
            aspects = (cls._general_aspects + cls._category_specific_aspects[row["appCategory"]]),
            aspects_and_scores = json.loads(row['aspectsAndScores']) if pd.isna(row['aspectsAndScores']) is False else None,
            generated_aspects_and_scores = generated_aspects_and_scores
        )

    def update_aspects_and_scores(self, column_name : str,  new_aspects_and_scores : Any, temp : Any = None, top_p : Any = None, predefined_aspects : bool = True, rationales : bool = False):
        if isinstance(new_aspects_and_scores, str):
          try:
              new_aspects_and_scores = json.loads(new_aspects_and_scores)
          except json.JSONDecodeError:
              raise ValueError("Invalid JSON format")
        if column_name not in self._generated_dataset_names:
              raise ValueError(f"Dataset name {column_name} is not in the list of allowed dataset names")
        if predefined_aspects and not rationales:
          for key, value in new_aspects_and_scores.items():
            if key not in self.aspects:
                raise ValueError(f"Aspect {key} is not in the list of aspects")
            if value not in self._valid_sentiments:
                raise ValueError(f"Sentiment {value} is not in the list of valid sentiments")
        if predefined_aspects and rationales:
          for key, value in new_aspects_and_scores.items():
            if key not in self.aspects:
                raise ValueError(f"Aspect {key} is not in the list of aspects")
            if value[0] not in self._valid_sentiments:
                raise ValueError(f"Sentiment {value} is not in the list of valid sentiments")
        if not predefined_aspects and not rationales:
          for key, value in new_aspects_and_scores.items():
            if value not in self._valid_sentiments:
                raise ValueError(f"Sentiment {value} is not in the list of valid sentiments")
        if not predefined_aspects and rationales:
          for key, value in new_aspects_and_scores.items():
            if value[0] not in self._valid_sentiments:
                raise ValueError(f"Sentiment {value} is not in the list of valid sentiments")
        if temp is not None:
          self.generated_aspects_and_scores[column_name] = {"temp": temp, "aspects_and_scores": new_aspects_and_scores}
        elif top_p is not None:
          self.generated_aspects_and_scores[column_name] = {"top_p": top_p, "aspects_and_scores": new_aspects_and_scores}


    @validator("app_name")
    def validate_app_name(cls, value):
        if value not in cls._unique_app_names:
            raise ValueError(f"App name {value} is not in the list of unique app names")
        return value

    @validator("app_category")
    def validate_app_category(cls, value):
        if value not in cls._unique_app_categories:
            raise ValueError(f"App category {value} is not in the list of unique app categories")
        return value

    @validator("aspects")
    def validate_aspects(cls, value, values):
        app_category = values["app_category"]
        valid_aspects = cls._general_aspects + cls._category_specific_aspects[app_category]
        for aspect in value:
            if aspect not in valid_aspects:
                raise ValueError(f"Aspect {aspect} is not in the list of valid aspects")
        return value

    @validator("aspects_and_scores")
    def validate_aspects_and_scores(cls, value, values):
        aspects_and_scores = value
        if aspects_and_scores is None:
          return aspects_and_scores
        field_dict = values
        aspects = field_dict["aspects"]
        for key, value in aspects_and_scores.items():
            if key not in aspects:
                raise ValueError(f"Aspect {key} is not in the list of aspects")
            if value not in cls._valid_sentiments:
                raise ValueError(f"Sentiment {value} is not in the list of valid sentiments")
        return aspects_and_scores

In [ ]:
OPENAI_API_KEY = userdata.get("OpenAI_API_KEY")
OPENAI_MODEL = 'gpt-3.5-turbo'
client = openai.Client(api_key=OPENAI_API_KEY)


In [ ]:
def generate_aspects_and_scores_json_format(review : Review, column_name, model_name=OPENAI_MODEL, temperature=None, top_p = None, prompt_raw = PROMPT_ABSA_few_shot, predefined_aspects=True, rationales = False):
  """Use this if model is 'gpt-3.5-turbo' or 'gpt-4-1106-preview'"""
  prompt = prompt_raw.format(review = review)
  if temperature is not None:
    response = client.chat.completions.create(
      model=model_name,  # Change this to "gpt-3.5-turbo"
      response_format={ "type": "json_object" },
      messages=[{"role": "user", "content": prompt}],
      max_tokens=800,  # Adjust as needed
      temperature=temperature,  # Adjust as needed
      n=1,  # Number of completions to generate
    )
  if top_p is not None:
    response = client.chat.completions.create(
      model=model_name,  # Change this to "gpt-3.5-turbo"
      response_format={ "type": "json_object" },
      messages=[{"role": "user", "content": prompt}],
      max_tokens=800,  # Adjust as needed
      top_p=top_p,  # Adjust as needed
      n=1,  # Number of completions to generate
    )
  new_aspects_and_scores = response.choices[0].message.content.lower()
  if predefined_aspects:
    try:
      review.update_aspects_and_scores(column_name=column_name,  new_aspects_and_scores=new_aspects_and_scores, temp=temperature, top_p=top_p, predefined_aspects = predefined_aspects, rationales = rationales)
    except ValueError as v:
      print(v)
      print(f"review : {review.body}")
      print(f"new_aspects_and_scores : {new_aspects_and_scores}")
      print(f"self.aspects : {review.aspects}")
      new_aspects_and_scores_dict = json.loads(new_aspects_and_scores)
      invalid_aspects = set(new_aspects_and_scores_dict.keys()) - set(review.aspects)
      print(f"The aspect(s) that was not found in self.aspects : {invalid_aspects}")
      cleaned_aspects_and_scores_dict = {key: value for key, value in new_aspects_and_scores_dict.items() if key not in invalid_aspects}
      review.update_aspects_and_scores(column_name=column_name,  new_aspects_and_scores=cleaned_aspects_and_scores_dict, temp=temperature, top_p=top_p, predefined_aspects=predefined_aspects, rationales=rationales)
  else:
      review.update_aspects_and_scores(column_name=column_name,  new_aspects_and_scores=new_aspects_and_scores, temp=temperature, top_p=top_p, predefined_aspects=predefined_aspects, rationales = rationales)
  print('success')

In [ ]:
# @title PROMPT_ABSA_zero_shot
PROMPT_ABSA_zero_shot = """Analyze and Annotate the following review with scores for the specific aspects (That are defined further below) for which a sentiment is expressed.

Return a JSON-Object where the keys are aspects and the values are the corresponding sentiment scores.

Sentiment Options: positive, negative, neutral

App Name: {review.app_name}
Review Title: \"\"\"{review.title}\"\"\"
Review Body: \"\"\"{review.body}\"\"\"
App Category : {review.app_category}
Use only one or more of the following Aspects : {review.aspects}

If an aspect in Aspects is not present in the review, omit it from the JSON-Object. If there are no clear aspects, return an empty JSON-Object. Keep aspects and sentiments in lower case."""

In [ ]:
# @title PROMPT_ABSA_few_shot
PROMPT_ABSA_few_shot = """Analyze and Annotate the following review with scores for the specific aspects (That are defined further below) for which a sentiment is expressed.

Return a JSON-Object where the keys are aspects and the values are the corresponding sentiment scores. Sentiment Scores should be either positive, negative or neutral.

App Name: {review.app_name}
Review Title: \"{review.title}\"
Review Body: \"{review.body}\"
App Category : {review.app_category}
Use only one or more of the following Aspects : {review.aspects}

Sentiment Options: positive, negative, neutral

#### Example 1:
- Review: \"This app has potential, but the lack of essential features like offline mode, dark mode, and collaboration options is disappointing. The features need improvement.\"
- Aspects and Sentiments : {{"satisfaction with features" : 'negative', "visual appeal and aesthetics" : 'negative', "offline access" : 'negative', "collaborative features" : 'negative'}}

#### Example 2:
- Review: \"While this app is decent, competitors like AppX and AppY offer a more seamless and efficient experience. They have better user interfaces and faster response times.\"
- Aspects and Sentiments: {{"competitive positioning" : 'negative', "satisfaction with features" : 'neutral', "user-friendly interface" : 'negative', "latency, framerate and reliability" : 'negative'}}

If an aspect in Aspects is not present in the review, omit it from the JSON-Object. If there are no clear aspects, return an empty JSON-Object."""

In [ ]:
# @title PROMPT_ABSA_explan
PROMPT_ABSA_explan = """Analyze and Annotate the following review with scores for the specific aspects (That are defined further below) for which a sentiment is expressed.

Return a JSON-Object where the keys are aspects and the values are the corresponding sentiment scores. Sentiment Scores should be either positive, negative or neutral.

App Name: {review.app_name}
Review Title: \"{review.title}\"
Review Body: \"{review.body}\"
App Category : {review.app_category}
Use one or more of the following Aspects : {review.aspects}

Sentiment Options: positive, negative, neutral

The aspect 'competitive positioning' should mainly be included and scored when the user mentions explicitly or implicitly that the app is better or worse than competiting apps
in which case the score should be 'positive' or 'negative' respectively

The aspect 'satisfaction with features' could be scored with 'negative' or 'neutral' if the user lists a number of feature improvements (even if the overall tone of the review is positive).

If an aspect in Aspects is not present in the review, omit it from the JSON-Object. If there are no clear aspects, return an empty JSON-Object. """

In [ ]:
# @title PROMPT_ABSA_experiment
PROMPT_ABSA_experiment = """Analyze and Annotate the following review with scores for the specific aspects (That are defined further below) for which a sentiment is expressed.

Return a JSON-Object where the keys are aspects and the values are the corresponding sentiment scores. Sentiment Scores should be either positive, negative or neutral.

App Name: {review.app_name}
Review Title: \"{review.title}\"
Review Body: \"{review.body}\"
App Category : {review.app_category}
Use one or more of the following Aspects : {review.aspects}

Sentiment Options: positive, negative, neutral

The aspect 'competitive positioning' should mainly be included and scored when the user mentions explicitly or implicitly that the app is better or worse or on par with competiting apps
in which case the score should be 'positive', 'negative' or 'neutral' respectively

The aspect 'satisfaction with features' could be scored with 'negative' or 'neutral' if the user lists a number of feature improvements (even if the overall tone of the review is positive).

#### Example 1:
- Review: \"This app has potential, but the lack of essential features like offline mode, dark mode, and collaboration options is disappointing. The features need improvement.\"
- Aspects and Sentiments : {{"satisfaction with features" : 'negative', "visual appeal and aesthetics" : 'negative', "offline access" : 'negative', "collaborative features" : 'negative'}}

#### Example 2:
- Review: \"While this app is decent, competitors like AppX and AppY offer a more seamless and efficient experience. They have better user interfaces and faster response times.\"
- Aspects and Sentiments: {{"competitive positioning" : 'negative', "satisfaction with features" : 'neutral', "user-friendly interface" : 'negative', "latency, framerate and reliability" : 'negative'}}

If an aspect in Aspects is not present in the review, omit it from the JSON-Object. But be generous and creative and include aspects even if they are only mentioned implicitly.  If there are no aspects, return an empty JSON-Object. """

In [ ]:
# @title PROMPT_ABSA_WITH_RATIONALES
PROMPT_ABSA_WITH_RATIONALES = """Consider the following review of a mobile application:

App Name: {review.app_name}
Review Title: \"{review.title}\"
Review Body: \"{review.body}\"
App Category : {review.app_category}

Perform aspect-based sentiment analysis on the review, and consider only the aspects explicitly or implicitly mentioned in the review.
Generate a JSON-string with aspects as keys and JSON-arrays as values with the sentiment polarities (positive, negative, neutral) as the first element and a short
rationale explaining what prompted the inclusion of each aspect as the second element.

#### Example 1:
- Review Title : \"Disappointing\"
- Review Body : \"This app has potential, but the lack of essential features like offline mode, dark mode, and collaboration options is disappointing. The features need improvement.\"
- Aspects, Sentiments and Rationales : {{
  "offline mode": ["negative", "The absence of offline mode is cited as a disappointment."],
  "dark mode": ["negative", "The lack of dark mode is missing and cited as a disappointment."],
  "collaboration options": ["negative", "The absence of collaboration options is cited as disappointing, suggesting dissatisfaction with the app's collaborative features."],
  "features": ["negative", "Overall dissatisfaction with the app's features, indicating that they need improvement to meet user expectations."]
}}

#### Example 2:
- Review Title: \"Its OK, but competitors are better\"
- Review Body: \"While this app is decent, competitors like AppX and AppY offer a more seamless and efficient experience. They have better user interfaces and faster response times.\"
- Aspects and Sentiments: {{
  "user interface": ["negative", "The comparison suggests that the app's user interface is inferior to competitors, indicating dissatisfaction with its design or usability."],
  "response time": ["negative", "The mention of competitors having faster response times implies dissatisfaction with the app's performance in this aspect."]
}}

### Example 3:
- Review Title: \"This app is awesome!\"
- Review Body: \"This app is awesome! I love it! I can't wait to use it!\"
- Aspects and Sentiments: {{
}}

Be creative and include aspects even if they are only mentioned implicitly. Keep aspects and sentiments in lower case. If there are no aspects, return an empty JSON-Object."""

In [ ]:
# @title PROMPT_ABSA_WITH_RATIONALES_PREDEF_MAX8
PROMPT_ABSA_WITH_RATIONALES_PREDEF_MAX8 = """Consider the following review of a mobile application:

App Name: {review.app_name}
Review Title: \"{review.title}\"
Review Body: \"{review.body}\"
App Category : {review.app_category}


Perform aspect-based sentiment analysis on the review, and consider only the aspects explicitly mentioned in the review.
Generate a JSON-string with aspects as keys and JSON-arrays as values with the sentiment polarities (positive, negative, neutral) as the first element and a short
rationale explaining what prompted the inclusion of each aspect as the second element. I want to emphasize: NEVER, never ever include an aspect that is not mentioned explicitly!

Use one or more of the following Aspects : {review.aspects}
But include at most 8 of them. If an aspect is not mentioned, do not include it and provide no rationale. Prioritize the most pronounced aspects.

The aspect 'competitive positioning' should mainly be included and scored when the user mentions explicitly that the app is better or worse or on par with competiting apps
in which case the score should be 'positive', 'negative' or 'neutral' respectively

The aspect 'satisfaction with features' could be scored with 'negative' or 'neutral' if the user lists a number of feature improvements (even if the overall tone of the review is positive).

#### Example 1:
- Review: \"This app has potential, but the lack of essential features like offline mode, dark mode, and collaboration options is disappointing. The features need improvement.\"
- Aspects and Sentiments : {{
  "satisfaction with features" : ['negative', 'The reviewer feels that many expected features are missing, hence a negative sentiment is given on this aspect'],
  "visual appeal and aesthetics" : ['negative', 'The lack of dark mode can be seen as flaw in the apps visual appeal, hence a negative sentiment is given on this aspect'],
  "offline access" : ['negative', 'The reviewer mentions that offline mode is missing, hence a negative sentiment is given on this aspect'],
  "collaborative features" : ['negative', 'The reviewer feels that collaboration options are mssing, and hence a negative sentiment is given on this aspect']}}

#### Example 2:
- Review: \"While this app is decent, competitors like AppX and AppY offer a more seamless and efficient experience. They have better user interfaces and faster response times.\"
- Aspects and Sentiments: {{
  "competitive positioning" : ['negative', 'The reviewer feel that the app is not on par with competitors, hence a negative sentiment is given on this aspect'],
  "satisfaction with features" : ['neutral', 'The reviewer mentions that the app is decent, but sees several improvements. Hence, a neutral sentiment is given'],
  "user-friendly interface" : ['negative', 'The reviewer feels that the apps competitors provide better user interfaces, hence a negative sentiment is given on this aspect'],
  "latency, framerate and reliability" : ['negative', 'The reviewer mentions that the apps competitors provide faster response times, hence a negative sentiment is given on this aspect']}}

If there are no aspects, return an empty JSON-Object."""

In [ ]:
# @title PROMPT_ABSA_WITH_RATIONALES_PREDEF_MAX8_IMP
PROMPT_ABSA_WITH_RATIONALES_PREDEF_MAX8_IMP = """Consider the following review of a mobile application:

App Name: {review.app_name}
Review Title: \"{review.title}\"
Review Body: \"{review.body}\"
App Category : {review.app_category}


Perform aspect-based sentiment analysis on the review, and consider only the aspects explicitly or implicity mentioned in the review.
Generate a JSON-string with aspects as keys and JSON-arrays as values with the sentiment polarities (positive, negative, neutral) as the first element and a short
rationale explaining what prompted the inclusion of each aspect as the second element.

Use one or more of the following Aspects : {review.aspects}
But include at most 8 of them. If an aspect is not mentioned, do not include it and provide no rationale. Prioritize the most pronounced aspects.

The aspect 'competitive positioning' should mainly be included and scored when the user mentions explicitly that the app is better or worse or on par with competiting apps
in which case the score should be 'positive', 'negative' or 'neutral' respectively

The aspect 'satisfaction with features' could be scored with 'negative' or 'neutral' if the user lists a number of feature improvements (even if the overall tone of the review is positive).

#### Example 1:
- Review: \"This app has potential, but the lack of essential features like offline mode, dark mode, and collaboration options is disappointing. The features need improvement.\"
- Aspects and Sentiments : {{
  "satisfaction with features" : ['negative', 'The reviewer feels that many expected features are missing, hence a negative sentiment is given on this aspect'],
  "visual appeal and aesthetics" : ['negative', 'The lack of dark mode can be seen as flaw in the apps visual appeal, hence a negative sentiment is given on this aspect'],
  "offline access" : ['negative', 'The reviewer mentions that offline mode is missing, hence a negative sentiment is given on this aspect'],
  "collaborative features" : ['negative', 'The reviewer feels that collaboration options are mssing, and hence a negative sentiment is given on this aspect']}}

#### Example 2:
- Review: \"While this app is decent, competitors like AppX and AppY offer a more seamless and efficient experience. They have better user interfaces and faster response times.\"
- Aspects and Sentiments: {{
  "competitive positioning" : ['negative', 'The reviewer feel that the app is not on par with competitors, hence a negative sentiment is given on this aspect'],
  "satisfaction with features" : ['neutral', 'The reviewer mentions that the app is decent, but sees several improvements. Hence, a neutral sentiment is given'],
  "user-friendly interface" : ['negative', 'The reviewer feels that the apps competitors provide better user interfaces, hence a negative sentiment is given on this aspect'],
  "latency, framerate and reliability" : ['negative', 'The reviewer mentions that the apps competitors provide faster response times, hence a negative sentiment is given on this aspect']}}

If there are no aspects, return an empty JSON-Object."""

In [ ]:
# @title PROMPT_ABSA_WITH_RATIONALES_uff
PROMPT_ABSA_WITH_RATIONALES_uff = """Consider the following review of a mobile application:

App Name: {review.app_name}
Review Title: "{review.title}"
Review Body: "{review.body}"
App Category : {review.app_category}

Perform aspect-based sentiment analysis on the review, and consider only the aspects explicitly or implicitly mentioned in the review.
Generate a JSON-string with aspects as keys and JSON-arrays as values with the sentiment polarities (positive, negative, neutral) as the first element and a short
rationale explaining what prompted the inclusion of each aspect as the second element. Please note that 'mixed' is not a valid sentiment, so do not ever include it.

I repeat, 'mixed' is not a sentiment. Use only 'positive', 'negative' or 'neutral' as the sentiment.


#### Example 1:
- Review Title : "Disappointing"
- Review Body : "This app has potential, but the lack of essential features like offline mode, dark mode, and collaboration options is disappointing. The features need improvement."
- Aspects, Sentiments and Rationales : {{
  "offline mode": ["negative", "The absence of offline mode is cited as a disappointment."],
  "dark mode": ["negative", "The lack of dark mode is missing and cited as a disappointment."],
  "collaboration options": ["negative", "The absence of collaboration options is cited as disappointing, suggesting dissatisfaction with the app's collaborative features."],
  "features": ["negative", "Overall dissatisfaction with the app's features, indicating that they need improvement to meet user expectations."]
}}

#### Example 2:
- Review Title: "Its OK, but competitors are better"
- Review Body: "While this app is decent, competitors like AppX and AppY offer a more seamless and efficient experience. They have better user interfaces and faster response times."
- Aspects and Sentiments: {{
  "user interface": ["negative", "The comparison suggests that the app's user interface is inferior to competitors, indicating dissatisfaction with its design or usability."],
  "response time": ["negative", "The mention of competitors having faster response times implies dissatisfaction with the app's performance in this aspect."]
}}

### Example 3:
- Review Title: "This app is awesome!"
- Review Body: "This app is awesome! I love it! I can't wait to use it!"
- Aspects and Sentiments: {{
}}

Be creative and include aspects even if they are only mentioned implicitly. Keep aspects and sentiments in lower case. If there are no aspects, return an empty JSON-Object. Please note that 'mixed' is not a valid sentiment, so do not ever include it."""

In [ ]:
# @title General aspects applicable to all or most apps
general_aspects = ['entertainment and fun',
                  'loading time(s)',
                  'user-friendly interface',
                  'developers\' responsiveness',
                  'cross-platform compatibility',
                  'offline access',
                  'notification management',
                  'data backup and restore',
                  'customization options',
                  'social media integration',
                  'collaborative features',
                  'search functionality',
                  'pricing and subscription plans',
                  'content variety',
                  'satisfaction with features',
                  'innovative features',
                  'special holiday content',
                  'content creation and customization',
                  'camera features',
                  'speaker and microphone features',
                  'latency, framerate and reliability',
                  'in-app community and engagement',
                  'latest update(s)',
                  'maintenance and downtime',
                  'in-app advertisements',
                  'visual appeal and aesthetics',
                  'customer support and assistance',
                  'battery usage efficiency',
                  'network and connectivity',
                  'personal view of company',
                  'competitive positioning',
                  'data privacy and security',
                  'integration with other apps']
len(general_aspects)

In [ ]:
# @title Category Specific App Categories for each category in the selected_reviews dataframe

category_specific_aspects = {'Lifestyle': ['wellness and health',
  'pregnancy, birth and postpartum tracking',
  'meeting new people and dating'],
 'Business': ['document management and scanning',
  'mobile workforce solutions',
  'pdf editing and ocr',
  'cloud integration for files',
  'expense tracking and reporting',
  'job and career resources'],
 'Education': ['interactive learning games',
  'creative exploration',
  'quizzes and assessments',
  'virtual learning environments',
  'classroom management tools',
  'language learning lessons',
  'math problem solving',
  'tutorials and videos'],
 'Entertainment': ['emoji and sticker creation',
  'movie listings and recommendations',
  'interactive video calls',
  'educational entertainment for kids',
  'animated hd backgrounds',
  'child-friendly video content',
  'video quality'],
 'Social Networking': ['custom emoji and stickers',
  'group video chatting',
  'premium features for social connections',
  'calls and texting',
  'instant messaging'],
 'Reference': ['slang and phrase definitions',
  'live voice and text translation',
  'advanced computational knowledge',
  'multilingual translation',
  'word meanings and definitions',
  'language reference and learning',
  'religious texts and study'],
 'Games': ['engaging gameplay mechanics',
  'immersive storytelling',
  'visual and audio experience',
  'multiplayer features',
  'innovative game design',
  'challenging levels and objectives',
  'user-friendly controls',
  'regular updates and new content',
  'community interaction and feedback',
  'microtransactions and in-app purchases',
  'performance and loading times'],
 'Sports': ['simulation and training',
  'coaching and strategy',
  'performance monitoring of teams and players',
  'event information and updates',
  'live sports updates',
  'fantasy sports management',
  'live sports streaming',
  'comprehensive sports coverage'],
 'Medical': ['accuracy of health information',
  'effective communication with healthcare providers',
  'integration with wearable devices',
  'accessibility and inclusivity',
  'pregnancy, birth and postpartum tracking'],
 'Photo & Video': ['photo editing and enhancement',
  'typography and artwork addition',
  'advanced photo editing features',
  'portrait retouching',
  'photo filters and effects',
  'ai-driven photo and video editing',
  'video editing and enhancement',
  'content discovery and streaming',
  'user-generated content features'],
 'Music': ['music discovery and exploration',
  'sheet music and notation',
  'tuning and metronome features',
  'dj and music mixing capabilities',
  'streaming quality and selection',
  'chords and tabs resources',
  'independent music upload and sharing',
  'integration with other apps',
  'playlist and library organization'],
 'News': ['content curation and presentation',
  'podcast player integration',
  'local news coverage',
  'opinion and analysis sections',
  'news source diversity and reliability'],
 'Weather': ['accuracy of weather predictions',
  'map and radar features',
  'hourly and daily forecast details',
  'location accuracy and customization',
  'severe weather alerts and information',
  'community and user reports'],
 'Health & Fitness': ['accuracy of health and fitness tracking',
  'workout plans and challenges',
  'meal tracking and nutrition information',
  'sleep tracking and meditation features',
  'pregnancy, birth and postpartum tracking'],
 'Utilities': ['predictive text capabilities', 'qr-scanning functionality'],
 'Finance': ['expense tracking and management',
  'debt management and payoff',
  'bills and payments organization',
  'transaction security and privacy',
  'payment and money transfer',
  'investment and savings tools',
  'budgeting capabilities'],
 'Shopping': ['product variety and selection',
  'price comparison and discounts',
  'shopping convenience and ease',
  'order tracking and delivery status',
  'secure payment options',
  'user reviews and ratings',
  'return and refund policy',
  'deals and promotions',
  'mobile shopping experience'],
 'Food & Drink': ['meal planning and preparation',
  'healthy eating options',
  'restaurant and delivery reviews',
  'food delivery speed and accuracy',
  'menu variety and options',
  'special offers and promotions',
  'overall dining experience'],
 'Navigation': ['accuracy of location services',
  'routing and navigation efficiency',
  'offline navigation capabilities',
  'real-time traffic updates',
  'points of interest information',
  'map display and clarity'],
 'Travel': ['flight status and tracking',
  'vehicle booking and ride-sharing',
  'navigation and maps',
  'marine traffic and ship tracking',
  'integration with other travel services',
  'overall travel experience'],
 'Book': ['audiobook collection and quality',
  'literary classics and poetry',
  'availability of free books',
  'library accessibility and borrowing',
  'overall reading and listening experience'],
 'Productivity': ['writing and editing tools',
  'focus and productivity enhancement',
  'dual screen and display options',
  'printing features and compatibility',
  'cloud storage and file management',
  'document collaboration and syncing',
  'email organization and management',
  'cloud drive integration']}

In [ ]:
completed_reviews_big = []

In [ ]:
nr_requests = 1000
batches_complete = 0.0

In [ ]:
for i in range(nr_requests*batches_complete, nr_requests*(batches_complete+1)): #
    print(f"row : {i}")
    review = Review.from_dataframe_row(selected_reviews.iloc[i])
    generate_aspects_and_scores_json_format(review, column_name='gpt-3.5-turbo-1106-temp-0.0-rationales', model_name='gpt-3.5-turbo', temperature=0.0, prompt_raw = PROMPT_ABSA_WITH_RATIONALES, predefined_aspects=False, rationales=True)
    completed_reviews_big.append(review)

batches_complete += 1

In [ ]:
i = None
print(f"row : {i}")
review = Review.from_dataframe_row(selected_reviews.iloc[i])
generate_aspects_and_scores_json_format(review, column_name='gpt-3.5-turbo-1106-temp-0.0-rationales', model_name='gpt-3.5-turbo', temperature=1.3, prompt_raw = PROMPT_ABSA_WITH_RATIONALES_uff, predefined_aspects=False, rationales=True)
completed_reviews_big.append(review)

In [ ]:
for review in completed_reviews_big:
  temp_a_s_r = review.generated_aspects_and_scores['gpt-3.5-turbo-1106-temp-0.0-rationales']['aspects_and_scores']
  temp_a_s = {}
  temp_rats = {}
  if len(list(temp_a_s_r.keys())) > 0:
    for aspect in temp_a_s_r:
      temp_a_s[aspect] = temp_a_s_r[aspect][0]
      temp_rats[aspect] = temp_a_s_r[aspect][1]
    selected_reviews.loc[selected_reviews['userReviewId'] ==review.user_review_id, 'ABSALabels'] = json.dumps(temp_a_s)
    selected_reviews.loc[selected_reviews['userReviewId'] ==review.user_review_id, 'ABSARationales'] = json.dumps(temp_rats)
  else:
    selected_reviews.loc[selected_reviews['userReviewId'] ==review.user_review_id, 'ABSALabels'] = json.dumps({})
    selected_reviews.loc[selected_reviews['userReviewId'] ==review.user_review_id, 'ABSARationales'] = json.dumps({})